In [1]:
import json

from sklearn.model_selection import train_test_split
import pandas as pd
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import math

from torch.utils.data import Dataset, DataLoader

## Load Data

In [2]:
file_path = '/content/spark_features.json'
with open(file_path, 'r') as file:
    data = json.load(file)

In [3]:
data

[['0QHEWicUbdyWuHsMcVd1Ka', '08Isz2ETWSBhvIl8UpKYsp'],
 ['0TdWXMXyfTQ2qp5vvU9LID', '0QHEWicUbdyWuHsMcVd1Ka'],
 ['0XwRlvv3KlOu4HWlOH34XG', '0TdWXMXyfTQ2qp5vvU9LID'],
 ['0biuGbhZwYnuUwMOi4fvaN', '0XwRlvv3KlOu4HWlOH34XG'],
 ['18kRElgBSIAsd3DHPUajuI', '0biuGbhZwYnuUwMOi4fvaN'],
 ['1YKPRycM3GzKbZUYDgLrmn', '18kRElgBSIAsd3DHPUajuI'],
 ['1jkNF7kVO0tzNiGXzpC6fQ', '1YKPRycM3GzKbZUYDgLrmn'],
 ['23NtsJjA1GhlKMAypBv9xR', '1jkNF7kVO0tzNiGXzpC6fQ'],
 ['2oenSXLDbWVaaL7QjSGYj5', '23NtsJjA1GhlKMAypBv9xR'],
 ['39QJ7KkN3L8LLqQqsO85W1', '2oenSXLDbWVaaL7QjSGYj5'],
 ['3F3ZuXYFXGM1V6pk3b3zmi', '39QJ7KkN3L8LLqQqsO85W1'],
 ['3GEiVoaUXq468gfYs2EPfh', '3F3ZuXYFXGM1V6pk3b3zmi'],
 ['3NwmvlTYDZPRBGDMGgJH8K', '3GEiVoaUXq468gfYs2EPfh'],
 ['3eoxzqrUUuGCTw8IUeeqcH', '3NwmvlTYDZPRBGDMGgJH8K'],
 ['3iNn2VvRJMpz5VLbEAH2r1', '3eoxzqrUUuGCTw8IUeeqcH'],
 ['3mZLiEDAHIjZ8ms5Lxq33F', '3iNn2VvRJMpz5VLbEAH2r1'],
 ['3s6bB02odIXvM3OpnqgzNN', '3mZLiEDAHIjZ8ms5Lxq33F'],
 ['3u6jK5KR1ijBCUs4CLzV9c', '3s6bB02odIXvM3OpnqgzNN'],
 ['42TMa2h

In [4]:
X = [d[0] for d in data]
y = [d[1] for d in data]
X

['0QHEWicUbdyWuHsMcVd1Ka',
 '0TdWXMXyfTQ2qp5vvU9LID',
 '0XwRlvv3KlOu4HWlOH34XG',
 '0biuGbhZwYnuUwMOi4fvaN',
 '18kRElgBSIAsd3DHPUajuI',
 '1YKPRycM3GzKbZUYDgLrmn',
 '1jkNF7kVO0tzNiGXzpC6fQ',
 '23NtsJjA1GhlKMAypBv9xR',
 '2oenSXLDbWVaaL7QjSGYj5',
 '39QJ7KkN3L8LLqQqsO85W1',
 '3F3ZuXYFXGM1V6pk3b3zmi',
 '3GEiVoaUXq468gfYs2EPfh',
 '3NwmvlTYDZPRBGDMGgJH8K',
 '3eoxzqrUUuGCTw8IUeeqcH',
 '3iNn2VvRJMpz5VLbEAH2r1',
 '3mZLiEDAHIjZ8ms5Lxq33F',
 '3s6bB02odIXvM3OpnqgzNN',
 '3u6jK5KR1ijBCUs4CLzV9c',
 '42TMa2hgBNjte4uV7jNCnQ',
 '4VosVUajzm6nzGlSb6nP3L',
 '4eBvRhTJ2AcxCsbfTUjoRp',
 '4fnIORP0QTYTOMPqMKpJtF',
 '53QrRlALFJfZgdcGLKwy5s',
 '5TjVLCESR9XIp5vDAWYfmi',
 '5ZLkihi6DVsHwDL3B8ym1t',
 '5bnfUWFS4sVeWx1JLA3K9Z',
 '5eGsHi6RcL33gb9k89UYaB',
 '610keNiNVTIkYD0CqeSerg',
 '64mInfKzmJIbuTvcnhRgfW',
 '6ZYxNjuAU9Vy3VtF6W1dtE',
 '6dgYHmc5yBnQFI4FSCbp6y',
 '72vuBPMhwFNlSYpTSf6fVD',
 '7d4ZrG3Cer7jbqIl6wOI6D',
 '7o1SDLghDwMc4OW3rKakVt']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = pd.DataFrame(data=X_train, columns=['prev_dish'])
X_test = pd.DataFrame(data=X_test, columns=['prev_dish'])
y_train = pd.DataFrame(data=y_train, columns=['next_dish'])
y_test = pd.DataFrame(data=y_test, columns=['next_dish'])

In [6]:
allDF = pd.DataFrame(data=data, columns=["prev_dish", "next_dish"])
allDF.sample(3)

,prev_dish,next_dish
10,3F3ZuXYFXGM1V6pk3b3zmi,39QJ7KkN3L8LLqQqsO85W1
26,5eGsHi6RcL33gb9k89UYaB,5bnfUWFS4sVeWx1JLA3K9Z
13,3eoxzqrUUuGCTw8IUeeqcH,3NwmvlTYDZPRBGDMGgJH8K


In [7]:
def encode_all(X_train, X_test, y_train, y_test):
    """
    Numeric Encoding of IDs.
    Function deals with unkown values in test by encoding them as zero.
    Inefficient, but gets the job done for now
    """
    # Thought itd be better to not have y encoding overlap with x (x has a 1, and y has 1, 2 etc)
    num_xSongs = len(X_train.prev_dish.unique())  # Variable for stopping y and x encoding from overlapping
    X_trainMap = {i+1:o for i,o in enumerate(X_train.prev_dish)}  # Create a map for encoding train data and checking test data
    X_train= pd.DataFrame(data=range(1, num_xSongs+1), columns=['prev_dish'])
    X_trainMap[0] = 'UNSEEN'

    for i in range(len(X_test)):
        if X_test.iloc[i][0] not in X_trainMap.values():  # If unseen, map to zero
            X_test.iloc[i][0] = 0
        else:
            X_test.iloc[i][0] = X_trainMap[X_test[i]]
    y_trainMap = {i+1:o for i,o in enumerate(y_train.next_dish)}
    y_train = pd.DataFrame(data=range(num_xSongs, num_xSongs + len(y_train)+1), columns=['next_dish'])
    y_trainMap[0] = 'UNSEEN'
    for i in range(len(y_test)):
        if y_test.iloc[i][0] not in y_trainMap.values():
            y_test.iloc[i][0] = 0
        else:
            y_test.iloc[i][0] = y_trainMap[y_test[i]]

    return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = encode_all(X_train, X_test, y_train, y_test)
trainComb = pd.concat([X_train, y_train], axis=1)
testComb = pd.concat([X_test, y_test], axis=1)

In [9]:
trainComb["prev_dish"].fillna(28, inplace=True)
trainComb.sample(3)

,prev_dish,next_dish
3,4.0,30
8,9.0,35
1,2.0,28


In [10]:
trainComb["prev_dish"] = trainComb["prev_dish"].astype(int)
testComb["prev_dish"] = testComb["prev_dish"].astype(int)
testComb["next_dish"] = testComb["next_dish"].astype(int)
trainComb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   prev_dish  28 non-null     int64
 1   next_dish  28 non-null     int64
dtypes: int64(2)
memory usage: 576.0 bytes


## Pytorch Dataset

In [11]:
y_train.shape

(28, 1)

In [12]:
class OurData(Dataset):
    def __init__(self, x, y):
        # x = torch.from_numpy(x) #
        # y = torch.from_numpy(y)
        self.x, self.y = x, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [13]:
xTrain_vals = trainComb["prev_dish"].values
yTrain_vals = trainComb["next_dish"].values
xTrain_vals = torch.Tensor(xTrain_vals).requires_grad_(True)
yTrain_vals = torch.Tensor(yTrain_vals).requires_grad_(True)

xTest_vals = testComb["prev_dish"].values
yTest_vals = testComb["next_dish"].values
xTest_vals = torch.Tensor(xTest_vals).requires_grad_(True)
yTest_vals = torch.Tensor(yTest_vals).requires_grad_(True)

In [14]:
train_ds = OurData(xTrain_vals, yTrain_vals)
valid_ds = OurData(xTest_vals, yTest_vals)

In [15]:
train_ds.x.shape, train_ds.y.shape

(torch.Size([28]), torch.Size([28]))

In [16]:
train_dl = DataLoader(train_ds, batch_size=7, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=7, shuffle=False)

## Modeling:

In [28]:
emb_size = 512
model = nn.Sequential(
        nn.Embedding(len(xTrain_vals)+1, emb_size),
        nn.ELU(),
        nn.Linear(emb_size, len(xTrain_vals)),  # output is a 28x28
        nn.Softmax(dim=1),
        )


In [29]:
def val_metric(model, valid_dl):
    model.eval()
    losses = []
    y_hats = []
    ys = []
    for x, y in valid_dl:  # Get batches from valid dl, but we use entire valdiation set
        y = y.unsqueeze(1)
        y_hat = model(x.long())
        y_hat = torch.argmax(y_hat, dim=1)
        loss = F.mse_loss(y_hat.float(), y.float())
        y_hats.append(y_hat.detach().numpy())
        ys.append(y.detach().numpy())
        losses.append(loss.item())

    ys = np.concatenate(ys)
    y_hats = np.concatenate(y_hats)
    return np.mean(losses)

In [41]:
## train_loop function
def train_loop(model, train_dl, valid_dl, optimizer, epochs):
    losses = []
    for i in range(epochs):
        model.train()
        for x, y in train_dl:  # Loop where X = 1 batch
            y = y.unsqueeze(1)
            y_hat = model(x.long())
            y_hat = torch.argmax(y_hat, dim=1)
            loss = F.mse_loss(y_hat.float(), y.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        train_loss = np.mean(losses)
        valid_loss = val_metric(model, valid_dl)  # Loops thru entire val set
        print(f"loss on epoch {i} = %.3f" %
              (train_loss))

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [42]:
train_loop(model, train_dl, valid_dl, optimizer, epochs=4)

loss on epoch 0 = 997.888
loss on epoch 1 = 995.969
loss on epoch 2 = 996.214
loss on epoch 3 = 991.742


<ipython-input-41-10a68b757db6>:10: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())


In [43]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
train_loop(model, train_dl, valid_dl, optimizer, epochs=4)

loss on epoch 0 = 994.745
loss on epoch 1 = 985.265
loss on epoch 2 = 987.204
loss on epoch 3 = 988.717


<ipython-input-41-10a68b757db6>:10: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())


In [45]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
train_loop(model, train_dl, valid_dl, optimizer, epochs=4)

loss on epoch 0 = 995.082
loss on epoch 1 = 994.638
loss on epoch 2 = 988.061
loss on epoch 3 = 988.449


<ipython-input-41-10a68b757db6>:10: UserWarning: Using a target size (torch.Size([7, 1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat.float(), y.float())
